In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from satellitepy.data.labels import read_label, read_fineair_label
from satellitepy.data.bbox import BBox
from satellitepy.utils.path_utils import get_file_paths
from satellitepy.data.utils import get_satellitepy_dict_values, count_unique_values

from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import cv2

# Analyze fuselage length and wingspan

In [59]:
def get_size_features(label_folder,label_format, spatial_resolution):
    fuselages = {'fine-class':{},'very-fine-class':{},'fineair-class':{}}
    wingspans = {'fine-class':{},'very-fine-class':{},'fineair-class':{}}
    label_paths = get_file_paths(label_folder)
    for label_path in label_paths:
        # label = read_fineair_label(label_path, include_fineair_class=True)
        label = read_label(label_path=label_path,label_format=label_format)
        obboxes = get_satellitepy_dict_values(label, 'obboxes')
        fine_classes = get_satellitepy_dict_values(label, 'fine-class')
        very_fine_classes = get_satellitepy_dict_values(label, 'very-fine-class')
        fineair_classes = get_satellitepy_dict_values(label, 'fineair-class') if 'fineair-class' in label.keys() else [None]*len(very_fine_classes) 
        for i, obbox in enumerate(obboxes):
            params = BBox(corners=obbox).params
            x, y, h, w, angle = params
            fine_class = fine_classes[i]
            very_fine_class = very_fine_classes[i]
            fineair_class = fineair_classes[i]

            # Fuselages
            if fine_class not in fuselages['fine-class'].keys():
                fuselages['fine-class'][fine_class] = []
            full_very_fine_class = f"{fine_class}-{very_fine_class}"
            if full_very_fine_class not in fuselages['very-fine-class'].keys():
                fuselages['very-fine-class'][full_very_fine_class] = []
            if fineair_class not in fuselages['fineair-class'].keys():
                fuselages['fineair-class'][fineair_class] = []
            fuselages['fine-class'][fine_class].append(h*spatial_resolution)
            fuselages['fineair-class'][fineair_class].append(h*spatial_resolution)
            fuselages['very-fine-class'][full_very_fine_class].append(h*spatial_resolution)

            # Wingspans
            if fine_class not in wingspans['fine-class'].keys():
                wingspans['fine-class'][fine_class] = []
    
            if full_very_fine_class not in wingspans['very-fine-class'].keys():
                wingspans['very-fine-class'][full_very_fine_class] = []

            if fineair_class not in wingspans['fineair-class'].keys():
                wingspans['fineair-class'][fineair_class] = []

            wingspans['fine-class'][fine_class].append(w*spatial_resolution)
            wingspans['very-fine-class'][full_very_fine_class].append(w*spatial_resolution)
            wingspans['fineair-class'][fineair_class].append(w*spatial_resolution)

    return wingspans, fuselages

In [5]:
def get_size_stats(wingspans, fuselages):
    size_stats = {'fine-class':{},'very-fine-class':{}}

    for class_i in ['fine-class', 'very-fine-class']:
        for key in wingspans[class_i].keys():
            size_stats[class_i][key] = [np.mean(wingspans[class_i][key]), 
                np.std(wingspans[class_i][key]),
                np.mean(fuselages[class_i][key]),
                np.std(fuselages[class_i][key])]
    return size_stats


In [71]:
%matplotlib
def plot_dist(wingspans_i,fuselages_i,class_name,class_type):
    
    # if class_name not in wingspans[class_type].keys():
    #     print(f'{class_name} does not exist in {class_type} field.')
    #     return 0
    # wingspans_i = np.array(wingspans[class_type][class_name])
    # fuselages_i = np.array(fuselages[class_type][class_name])
    
    ws_bins = [a for a in range(np.amin(wingspans_i).astype(int)-3,np.amax(wingspans_i).astype(int)+4,1)]
    fl_bins = [a for a in range(np.amin(fuselages_i).astype(int)-3,np.amax(fuselages_i).astype(int)+4,1)]

    fig, ax = plt.subplots(1,2)
    # ax.grid(zorder=0)
    ax[0].hist(wingspans_i,ws_bins)
    ax[1].hist(fuselages_i,fl_bins)
    ax[0].set_title(f'Wingspan Dist. of {class_name}')
    ax[1].set_title(f'Length Dist. of {class_name}')
    # plt.xticks(bins)
    output_path = f'/home/murat/Projects/satellitepy/docs/temp_{class_name}.png'
    print(f"Figure is saved here: {output_path}")
    plt.savefig(output_path)
    # plt.show()
    plt.close()

Using matplotlib backend: QtAgg


## Plot histogram

In [70]:
label_folder = Path("/home/murat/Projects/satellitepy/data/fineair/labels_fineair/labels")
label_format = 'fineair' # 
spatial_resolution = 0.3

wingspans, fuselages = get_size_features(label_folder,label_format,spatial_resolution)
# size_stats = get_size_stats(wingspans,fuselages)
# class_size_stats = size_stats['very-fine-class']
class_type='fineair-class'
class_name='B787'
plot_dist(wingspans[class_type][class_name],fuselages[class_type][class_name],class_name,class_type)
# print(size_stats)

# for key, values in class_size_stats.items():
#     value_to_print = [f"{item:.1f}" for item in values]
#     print(f"{key}: {', '.join(value_to_print)}")
# print(wingspans)
# print(fuselages)
# print([key for key in wingspans.keys() if len(wingspans[key])>20])

Figure is saved here: /home/murat/Projects/satellitepy/docs/temp_B787.png


# Clusters

In [60]:
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

def fit_gmm(sizes, max_components=3):
    """
    Fit a Gaussian Mixture Model to the data and determine the optimal number of components using BIC.

    Parameters:
    - sizes: A 2D array of shape (n_samples, n_features) containing the size data (e.g., length, width).
    - max_components: The maximum number of components to try.

    Returns:
    - gmm: The fitted Gaussian Mixture Model with the optimal number of components.
    - bic: The BIC values for each number of components.
    """
    bics = []
    models = []
    for n in range(1, max_components + 1):
        gmm = GaussianMixture(n_components=n, random_state=42)
        gmm.fit(sizes)
        bics.append(gmm.bic(sizes))
        models.append(gmm)
    
    # Determine the optimal number of components
    optimal_n_components = np.argmin(bics) + 1
    optimal_gmm = models[optimal_n_components - 1]

    # Plot BIC values
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, max_components + 1), bics, marker='o')
    plt.title('BIC vs. Number of Components')
    plt.xlabel('Number of Components')
    plt.ylabel('BIC')
    plt.grid(True)
    plt.show()

    print(f'Optimal number of components: {optimal_n_components}')
    return optimal_gmm, bics

In [67]:
from scipy.stats import zscore

def remove_outliers_zscore(data, threshold=3):
    z_scores = np.abs(zscore(data))
    filtered_entries = (z_scores < threshold).all(axis=1)
    return data[filtered_entries]

In [ ]:
wingspans, fuselages = get_size_features(label_folder)

In [77]:
class_name='A320'
class_type = 'fine-class'
print(len(wingspans[class_type][class_name]))
sizes = np.array([[w,f] for w, f in zip(wingspans[class_type][class_name],fuselages[class_type][class_name])])
sizes = remove_outliers_zscore(sizes)
# print(len(wingspans[class_type][class_name]))

plot_dist(sizes[:,0],sizes[:,1],class_name,class_type)
fit_gmm(sizes, max_components=10)

320
Optimal number of components: 4


(GaussianMixture(n_components=4, random_state=42),
 [2247.2174924920932,
  2056.990999998179,
  2059.3316641025062,
  2049.178310792422,
  2080.750350803246,
  2084.23515037484,
  2097.1268987108715,
  2130.9291736548053,
  2162.61409353253,
  2154.715789604004])

# Get contours of the mask templates

In [ ]:
mask_template_dir = Path('/home/murat/Projects/satellitepy/data/TransAir3M/mask_templates')
out_dir = Path('/home/murat/Projects/satellitepy/data/TransAir3M/mask_template_contours')
mask_paths = get_file_paths(mask_template_dir)

for mask_path in mask_paths:
    img = cv2.imread(str(mask_path),1)
    img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    _,img_binary = cv2.threshold(img_gray,50,255,cv2.THRESH_BINARY)

    img_contours = np.zeros_like(img_binary)
    contours, hierarchy = cv2.findContours(img_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    cv2.drawContours(img_contours, contours, -1, 255, 1)

    # img_contours = cv2.cvtColor(img_contours,cv2.COLOR_BGR2BGRA)
    zeros_channel = np.zeros_like(img_binary)
    bgra = [zeros_channel,zeros_channel,img_contours,img_contours]
    img_contours_alpha = cv2.merge(bgra,4)
    # print(np.nonzero(img_contours_alpha))
    # print(img_contours_alpha.shape)

    out_mask_path = str(out_dir / f"{mask_path.stem}.png")
    cv2.imwrite(out_mask_path, img_contours_alpha) 
    

# Print annotation source ratio for each label file

In [ ]:
task = 'source'
label_folder = Path("/mnt/2tb-1/satellitepy/data/FR24_dataset/labels") # Path("/home/murat/Projects/satellitepy/data/fr24/labels_all")
label_format = 'fr24' # 'satellitepy' # 
label_paths = get_file_paths(label_folder)

for label_path in label_paths:
    print(label_path.name)
    instances = {}
    label = read_label(label_path,label_format)
    values = get_satellitepy_dict_values(label, task)
    unique_values = count_unique_values(values,instances=instances)
    if 'FR24' in unique_values:
        fr24_ratio = unique_values['FR24']/sum(unique_values.values())
        print(unique_values, f"fr24 ratio is {fr24_ratio:.2f}")

# Read IMD files

In [23]:
import re
import os
import csv
def parse_imd_file(file_content):
    # Define regex patterns to match the start and end of the group, and key-value pairs
    group_start_pattern = r'BEGIN_GROUP\s*=\s*IMAGE_1'
    group_end_pattern = r'END_GROUP\s*=\s*IMAGE_1'
    key_value_pattern = r'(\w+)\s*=\s*("[^"]*"|[^;]+);'


    # Patterns for band groups
    band_patterns = {
        "BAND_B": r'BEGIN_GROUP\s*=\s*BAND_B(.*?)END_GROUP\s*=\s*BAND_B',
        "BAND_G": r'BEGIN_GROUP\s*=\s*BAND_G(.*?)END_GROUP\s*=\s*BAND_G',
        "BAND_R": r'BEGIN_GROUP\s*=\s*BAND_R(.*?)END_GROUP\s*=\s*BAND_R',
        "BAND_N": r'BEGIN_GROUP\s*=\s*BAND_N(.*?)END_GROUP\s*=\s*BAND_N'
    }
    bandwidth_pattern = r'effectiveBandwidth\s*=\s*([\d\.e\+\-]+);'

    # Fields of interest
    fields_of_interest = {
        "satId": str,
        "minCollectedRowGSD": float,
        "maxCollectedRowGSD": float,
        "minSunAz": float,
        "maxSunAz": float,
        "minSunEl": float,
        "maxSunEl": float,
        "minSatAz": float,
        "maxSatAz": float,
        "minSatEl": float,
        "maxSatEl": float,
        "minOffNadirViewAngle": float,
        "maxOffNadirViewAngle": float,
        "cloudCover": float,
        "resamplingKernel": str,
        "BAND_B": float,
        "BAND_G": float,
        "BAND_R": float,
        "BAND_N": float
    }

    # Find the IMAGE_1 group content
    group_content = re.search(f'{group_start_pattern}(.*?){group_end_pattern}', file_content, re.DOTALL)
    if not group_content:
        return None

    group_content = group_content.group(1)

    # Extract key-value pairs within the group
    metadata = {}
    for match in re.finditer(key_value_pattern, group_content):
        key, value = match.groups()
        if key in fields_of_interest:
            value = value.strip().strip('"')
            expected_type = fields_of_interest[key]
            if expected_type == float:
                value = float(value)
            elif expected_type == int:
                value = int(value)
            metadata[key] = value

    # Extract bandwidths for each band
    for band, pattern in band_patterns.items():
        band_content = re.search(pattern, file_content, re.DOTALL)
        if band_content:
            bandwidth_match = re.search(bandwidth_pattern, band_content.group(1))
            if bandwidth_match:
                metadata[band] = float(bandwidth_match.group(1))
        else:
            metadata[band] = None

    return metadata

def parse_multiple_imd_files(parent_directory):
    aggregated_data = {
        "filePath": [],
        "satId": [],
        "minCollectedRowGSD": [],
        "maxCollectedRowGSD": [],
        "minSunAz": [],
        "maxSunAz": [],
        "minSunEl": [],
        "maxSunEl": [],
        "minSatAz": [],
        "maxSatAz": [],
        "minSatEl": [],
        "maxSatEl": [],
        "minOffNadirViewAngle": [],
        "maxOffNadirViewAngle": [],
        "cloudCover": [],
        "resamplingKernel": [],
        "BAND_B": [],
        "BAND_G": [],
        "BAND_R": [],
        "BAND_N": []
    }

    for root, _, files in os.walk(parent_directory):
        for filename in files:
            if filename.endswith(".IMD"):
                file_path = os.path.join(root, filename)
                with open(file_path, 'r') as file:
                    file_content = file.read()
                metadata = parse_imd_file(file_content)
                if metadata:
                    aggregated_data["filePath"].append(file_path)
                    for key in aggregated_data:
                        if key != "filePath" and key in metadata:
                            aggregated_data[key].append(metadata[key])

    return aggregated_data


# Parent directory containing IMD files and subdirectories
parent_directory = '/mnt/2tb-1/satellitepy/data/'

# Parse the files and get the aggregated data
aggregated_data = parse_multiple_imd_files(parent_directory)
print(aggregated_data['BAND_N'])
# print(len(aggregated_data['filePath']))

def write_to_csv(data, csv_filename):
    # Get the keys for the CSV columns
    keys = data.keys()

    # Write the dictionary to a CSV file
    with open(csv_filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=keys)
        writer.writeheader()
        
        # Get the number of rows
        num_rows = len(data["filePath"])

        # Write the data row by row
        for i in range(num_rows):
            row = {key: data[key][i] for key in keys}
            writer.writerow(row)


csv_path = '/home/murat/Projects/satellitepy/docs/fineair_metadata_raw.csv'
write_to_csv(aggregated_data, csv_path)

[0.1004, 0.0989, 0.0989, 0.0989, 0.0989, 0.1004, 0.1004, 0.0989, 0.1004, 0.1004, 0.0989, 0.0989, 0.0989, 0.1004, 0.1004, 0.0989, 0.0989, 0.0989, 0.14, 0.1004, 0.0989, 0.14, 0.0989, 0.1004, 0.1004, 0.1004, 0.14, 0.1004, 0.0989, 0.1004, 0.14, 0.1004, 0.1004, 0.14, 0.1004, 0.1004, 0.1004, 0.0989, 0.14, 0.1004, 0.1004, 0.0989, 0.0989, None, 0.0989, 0.14, 0.14, 0.1004, 0.1004, 0.0989, 0.1004, 0.1004]


# Print paths from given fine class

In [ ]:
###
my_fine_class = 'B787'
my_fineair_class = None
my_very_fine_class = None
###
my_label_folder = Path('/home/murat/Projects/satellitepy/data/fineair/labels_fineair/labels')
label_format = 'fineair'

label_paths = get_file_paths(my_label_folder)
for label_path in label_paths:
    # label = read_fineair_label(label_path, include_fineair_class=False)
    label = read_label(label_path, label_format)
    # obboxes = get_satellitepy_dict_values(label, 'obboxes')
    fine_classes = get_satellitepy_dict_values(label, 'fine-class')
    fineair_classes = get_satellitepy_dict_values(label, 'fineair-class')
    very_fine_classes = get_satellitepy_dict_values(label, 'very-fine-class')
    if my_fineair_class:
        for i, fineair_class in enumerate(fineair_classes):
            if fineair_class == my_fineair_class:
                print(f'{my_fineair_class} found! Subtype is {very_fine_classes[i]}')
                print(label_path)
    if my_fine_class:
        for i, fine_class in enumerate(fine_classes):
            if fine_class == my_fine_class:
                print(f'{my_fine_class} found! Subtype is {very_fine_classes[i]}')
                print(label_path)


# FAIR1M

In [74]:
## Plot histogram for FAIR1M
label_folder = Path("/mnt/2tb-0/satellitepy/data/Gaofen_merged/val/labels/")
label_format = 'satellitepy' # 
spatial_resolution = 0.6
wingspans, fuselages = get_size_features(label_folder,label_format,spatial_resolution)
# size_stats = get_size_stats(wingspans,fuselages)
# class_size_stats = size_stats['very-fine-class']
class_type='fine-class'
class_name='Boeing787'
plot_dist(wingspans[class_type][class_name],fuselages[class_type][class_name],class_name,class_type)

Figure is saved here: /home/murat/Projects/satellitepy/docs/temp_Boeing787.png


In [56]:
##
my_fine_class = 'Boeing787'
###
my_label_folder = Path("/mnt/2tb-0/satellitepy/data/Gaofen_merged/val/labels/")
label_format = 'satellitepy'

label_paths = get_file_paths(my_label_folder)
for label_path in label_paths:
    label = read_label(label_path,label_format)
    fine_classes = get_satellitepy_dict_values(label, 'fine-class')
    if my_fine_class:
        for i, fine_class in enumerate(fine_classes):
            if fine_class == my_fine_class:
                print(f'{my_fine_class} found!')
                print(label_path)

Boeing787 found!
/mnt/2tb-0/satellitepy/data/Gaofen_merged/val/labels/0.json
Boeing787 found!
/mnt/2tb-0/satellitepy/data/Gaofen_merged/val/labels/0.json
Boeing787 found!
/mnt/2tb-0/satellitepy/data/Gaofen_merged/val/labels/0.json
Boeing787 found!
/mnt/2tb-0/satellitepy/data/Gaofen_merged/val/labels/0.json
Boeing787 found!
/mnt/2tb-0/satellitepy/data/Gaofen_merged/val/labels/1.json
Boeing787 found!
/mnt/2tb-0/satellitepy/data/Gaofen_merged/val/labels/1.json
Boeing787 found!
/mnt/2tb-0/satellitepy/data/Gaofen_merged/val/labels/1.json
Boeing787 found!
/mnt/2tb-0/satellitepy/data/Gaofen_merged/val/labels/1.json
Boeing787 found!
/mnt/2tb-0/satellitepy/data/Gaofen_merged/val/labels/1.json
Boeing787 found!
/mnt/2tb-0/satellitepy/data/Gaofen_merged/val/labels/10.json
Boeing787 found!
/mnt/2tb-0/satellitepy/data/Gaofen_merged/val/labels/10.json
Boeing787 found!
/mnt/2tb-0/satellitepy/data/Gaofen_merged/val/labels/10.json
Boeing787 found!
/mnt/2tb-0/satellitepy/data/Gaofen_merged/val/labels/10.